In [16]:
# Import Dependencies
import pandas as pd
from pathlib import Path
import requests

# Import the OpenExchangeRates API key
from api_keys import openexchangerates_api_key

In [17]:
customer_shopping_data_csv = Path("./Resources/source/customer_shopping_data.csv")

customer_shopping_data_df = pd.read_csv(customer_shopping_data_csv)
customer_shopping_data_df.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,9/11/2021,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM
4,I337046,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon


In [20]:
#Convert the date from a string to the datetime type using pd.to_datetime, specifying the source format as day/month/year, then convert to date using .dt.date
#This is necessary to ensure the date is in a format compatible with the API.

df = pd.to_datetime(customer_shopping_data_df["invoice_date"], format="%d/%m/%Y").dt.date
df = df.drop_duplicates(keep="first")
df = df.sort_values()

df.head()

4296    2021-01-01
1261    2021-01-02
220     2021-01-03
551     2021-01-04
1648    2021-01-05
Name: invoice_date, dtype: object

In [15]:
#Set the API base URL
base_url = "https://openexchangerates.org/api/historical"

date = []
exchange_rate = []

#Loop through all dates. Currently, a limit is set to the first 3 dates to avoid using up the free API requests.
#It's better to conduct experiments on 2-3 dates to test the loop and ensure that the API returns varied (and accurate) information for each date.
#For a production launch, use for invoice_date in df:

for invoice_date in df.head(3):

    #Assemble the final string for the API with all parameters.
    url = f"{base_url}/{invoice_date}.json?app_id={openexchangerates_api_key}&base=USD&symbols=TRY"

    #Implement logging that is convenient for visual monitoring, as the full run takes 7-10 minutes.
    print(f"currently processing {invoice_date}...")

    # Make the API request
    response = requests.get(url)
        
    # Convert response to JSON
    data = response.json()
    date.append(invoice_date)
    exchange_rate.append(data["rates"]["TRY"])

currently processing 2021-01-01...
currently processing 2021-01-02...
currently processing 2021-01-03...


In [8]:
exchange_rate_df = pd.DataFrame({
    "date": date,
    "exchange_rate": exchange_rate
})

exchange_rate_df.head()

,date,exchange_rate
0,2021-01-01,7.439200
1,2021-01-02,7.439200
2,2021-01-03,7.441906
3,2021-01-04,7.427000
4,2021-01-05,7.390200


In [9]:
exchange_rate_df.to_csv("./Resources/output/exchange_rate.csv", header=True, index=False)